In [1]:
import torch
from circuits import Circuit,Kinds
from learn import Trainer
from data import Preprocess

In [2]:
# Get cpu or gpu device for training.
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

# torch.autograd.set_detect_anomaly(True)
torch.set_printoptions(precision=3, linewidth=200)

Using cpu device


In [3]:
vr = Circuit()
vr.ring(Kinds.IVS,Kinds.R,2)
vr.elements[0].attr = 1e7
vr.elements[-1].v = 1

In [4]:
input = Preprocess(vr)
trainer = Trainer(input)

1e-05


c:\Users\terry\OneDrive\Documents\GitHub\side_circuit\ml\circuits.py:104: FutureWarning: incidence_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  M_scipy = nx.incidence_matrix(G=self.nx_graph(),oriented=True)


In [5]:
trainer.run(1000)

init target: [1.0, 1.0, 1.0, 1.0, 1e-07, 1e-07, 1.0, 1.0, 1.0]
init params: Parameter containing:
tensor([1.000e+00, 1.010e-05, 1.000e-12], requires_grad=True)
init loss: 0.12499995529651642
Done! at 258 passes, 3.1411563780138595e-21 loss
i, v, attr
tensor([[-1.662e+04,  1.000e+07,  1.000e+07],
        [ 1.662e+04,  1.000e+07,  6.016e+02],
        [ 1.662e+04,  1.000e+00,  6.015e-05]], grad_fn=<CatBackward0>)
